# Harmonic Spotify playlists
This creates harmonically ordered playlists from the clustered songs

## Import modules

In [1]:
import math
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances, pairwise
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from IPython.core.display import HTML
from IPython.display import IFrame
from sklearn.decomposition import PCA
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import matplotlib.colors as mcolors
import dotenv
import os
import base64
try:
  import spotipy
except:
  %pip install spotipy
  import spotipy
import pickle
from spotipy.oauth2 import SpotifyOAuth


In [2]:
for _ in [ '../src' ]:
  if not _ in sys.path:
    sys.path.append(_)

In [3]:
from importlib import reload
try:
  reload(sys.modules['harmonic_distance'])
except:
  pass
from harmonic_distance import key_aware_pairwise_distances, harmonic_scale

## Setup

In [4]:
from sklearn import set_config
set_config(transform_output="pandas")

## Read songs and cluster labels

In [5]:
spotify_df = (
  pd.read_csv('../data/6.3.3_spotify_5000_songs.csv', index_col=0)
  .drop_duplicates()
  .rename(columns=lambda x: x.strip())
  .assign(id=lambda x: x.id.str.strip())
)

In [6]:
clean_df = spotify_df[lambda x: x.time_signature != 0]

In [7]:
label_df = pd.read_csv('../data/6.3.3_spotify_5000_songs_labels.csv', index_col=0)
label_df

,label,distance
index,,
3677,0,0.141009
4050,0,0.155900
3549,0,0.167089
3765,0,0.170158
3649,0,0.170612
...,...,...
1702,5,1.168707
1689,5,1.188530
4091,5,1.265998


## Order top-50 songs of each cluster by harmony

In [8]:
harmonic_scaled_df = (
        harmonic_scale(clean_df, normalized=False, unwrap=False)
        .join(label_df)
        .sort_values(['label', 'distance'])
        .groupby('label')
        .head(50)
        .reset_index()
        .set_index(['label', 'harmonic'])
        .sort_values(['label', 'harmonic', 'distance'])
)
harmonic_scaled_df

index                                               name  \
label harmonic                                                             
0     -5         3649  Embalmed Alive                                ...   
      -5         3957  Dark Eternity                                 ...   
      -5         3872  Pestilential Mists                            ...   
      -5         3712  Confessions of a Serial Killer                ...   
      -5         4072  Foetal Carnage                                ...   
...               ...                                                ...   
5      5         3503  Addicted to Vaginal Skin                      ...   
       6         3046  Rage Valley                                   ...   
       6         2963  Arcade - Radio Edit                           ...   
       6         2976  The House Of House                            ...   
       6         4322  Turning Back - remastered 2003                ...   

                                                          artist  \
label harmonic                                                     
0     -5        Mortician                                          
      -5        Therion                                            
      -5        Abhorrence                                         
      -5        Gorefest                                           
      -5        Gorefest                                           
...                                                          ...   
5      5        Cannibal Corpse                                    
       6        Knife Party                                        
       6        Dimitri Vegas & Like Mike                          
       6        Dimitri Vegas & Like Mike                          
       6        Grave                                              

                danceability  energy  loudness  mode  speechiness  \
label harmonic                                                      
0     -5               0.309   0.915   -10.035     1       0.0988   
      -5               0.225   0.811    -8.946     1       0.0877   
      -5               0.183   0.945    -8.010     1       0.1160   
      -5               0.206   0.933    -5.952     1       0.1200   
      -5               0.237   0.978    -4.509     1       0.1210   
...                      ...     ...       ...   ...          ...   
5      5               0.244   0.987    -7.661     0       0.1230   
       6               0.434   0.962    -2.890     0       0.1260   
       6               0.565   0.957    -4.167     0       0.0723   
       6               0.565   0.892    -5.221     0       0.0672   
       6               0.265   0.823    -5.918     0       0.1090   

                acousticness  instrumentalness  liveness  valence    tempo  \
label harmonic                                                               
0     -5            0.000329             0.673    0.2710   0.0790  115.663   
      -5            0.000002             0.638    0.0598   0.3240  112.476   
      -5            0.000005             0.734    0.0664   0.2800  132.177   
      -5            0.000010             0.772    0.0568   0.1850  120.012   
      -5            0.000003             0.799    0.0325   0.2550  115.225   
...                      ...               ...       ...      ...      ...   
5      5            0.000057             0.779    0.0664   0.2470  142.341   
       6            0.000300             0.734    0.1050   0.2260  128.162   
       6            0.009230             0.690    0.2930   0.1750  127.953   
       6            0.144000             0.801    0.0385   0.0764  139.957   
       6            0.000003             0.777    0.1150   0.5110  136.107   

                 type  duration_ms  time_signature                      id  \
label harmonic                                                               
0     -5                     52533               4  7J4epP4wHOT7ta4CMshfk

## Harmonic random walk through each top-50 cluster

In [9]:
import random

def make_harmonic_playlist(df):
  def wrap_harmonic(harmonic, delta = 0):
    return (harmonic + delta + 5) % 12 - 5

  df = df.copy().reset_index().set_index(['harmonic', 'index'], drop=False).drop(columns=['index'])

  current_harmony = random.randint(-5, 6)
  current_direction = random.choice([-1, 1])

  result = None

  while len(df):
    while True:
      try:
        picked = df.loc[current_harmony].sample(1)
      except:
        current_harmony = wrap_harmonic(current_harmony, current_direction)
        continue
      break
    df.drop((current_harmony, picked.index[0]), inplace=True)
    result = pd.concat([result, picked])
    if random.random() < 0.5:
      current_direction *= -1
    if random.random() < 0.5:
      current_harmony = wrap_harmonic(current_harmony, current_direction)
  return result

cluster_playlists = { label: make_harmonic_playlist(harmonic_scaled_df.loc[label]) for label in harmonic_scaled_df.index.levels[0] }


## Import to Spotify via `spotipy` API

Save your [Spotify API](https://developer.spotify.com/) credentials and username from [your profile](https://www.spotify.com/account/profile/) in `.env` as:

```
SPOTIFY_CLIENT_ID=<YOUR_ID>
SPOTIFY_CLIENT_SECRET=<YOUR_SECRET>
SPOTIFY_USERNAME=<YOUR_USERNAME>
```

Use `http://localhost:8142/` as the redirection URL when setting up your app in the API dashboard.

In [10]:
dotenv.load_dotenv()

True

In [11]:
spotifyOAuth = SpotifyOAuth(
  client_id = os.getenv('SPOTIFY_CLIENT_ID'),
  client_secret = os.getenv('SPOTIFY_CLIENT_SECRET'),
  redirect_uri='http://localhost:8142/',
  scope='playlist-modify-private,ugc-image-upload',
  username=os.getenv('SPOTIFY_USERNAME'),
)
spotify = spotipy.Spotify(auth_manager=spotifyOAuth)

In [12]:
import json

playlist_names = [
  'WBSCS G5 Thunderous Riffs',
  'WBSCS G5 Feelgood Frequencies',
  'WBSCS G5 Groovy Pop Blend',
  'WBSCS G5 Study Focus',
  'WBSCS G5 Bars & Beats',
  'WBSCS G5 Death Metal Essentials'
]

try:
  playlist_ids = json.load(open('playlist_ids.json', 'r'))
except:
  playlist_ids = {}

playlist_ids

{'1': '6uM33lzLVKtQUr7HVXE4El',
 '0': '6HQlDhCiT48pZND5b8X3Py',
 '2': '7blNRyyIQnRqj8Cwksis5C',
 '3': '2S9hpuyFWv0tbAiCoX9qmC',
 '4': '2UfeQYHIX57ycihZgX73Fg',
 '5': '7CcAmzpv4KMjmrY3Ax43vq'}

In [76]:
import urllib.parse

for label in cluster_playlists.keys():
  try:
    playlist_id = playlist_ids[label]
  except:
    playlist_id = spotify.user_playlist_create(
      user=os.getenv('SPOTIFY_USERNAME'),
      name=playlist_names[label],
      public=False)['id']
    playlist_ids[label] = playlist_id

  with open(f'../images/playlist_cover_images/cover_{label}.jpg', 'rb') as image_file:
    image_b64 = base64.b64encode(image_file.read())

  spotify.playlist_upload_cover_image(
    playlist_id,
    image_b64
  )

  spotify.playlist_change_details(
    playlist_id,
    description='\u2764 Harmony-mixed playlist! \U0001F39A Set cross-fading to 12 seconds for best results!'
  )

  spotify.playlist_replace_items(
    playlist_id=playlist_id,
    items=[
      f'spotify:track:{str(id).strip()}'
      for id in cluster_playlists[label].id
    ]
  )

In [77]:
json.dump(playlist_ids, open('playlist_ids.json', 'w'))

## Display of the playlists

In [38]:
def make_spotify_link(url):
    return f'<a href="{url}">Play</a>'

def embed_spotify(trackid):
  return f'<iframe style="border-radius:12px" src="https://open.spotify.com/embed/track/{trackid.strip()}?utm_source=generator" width="350" height="152" frameBorder="0" allowfullscreen="" allow="autoplay; clipboard-write; encrypted-media; fullscreen; picture-in-picture" loading="lazy"></iframe>'

def embed_spotify_button_only(trackid):
  return f'''<div style="overflow:hidden; border-radius:12px; border-style:hidden; width:48px; height:48px"
    ><iframe src="https://open.spotify.com/embed/track/{trackid.strip()}?utm_source=generator"
            width="300"
            height="80"
            style="border-radius:12px; position:relative; left:-252px; top:-32px;"
            frameBorder="0"
            allowfullscreen=""
            allow="autoplay; clipboard-write; encrypted-media; fullscreen; picture-in-picture"
            loading="lazy"
    ></iframe></div>
    '''
HTML(embed_spotify(spotify_df.iloc[0].id))

/home/til/local/opt/anaconda3/lib/python3.11/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [39]:
colormap = px.colors.qualitative.Plotly

In [ ]:
(
  cluster_playlists[4][['harmonic', 'mode', 'id', 'artist', 'name']].iloc[0:10]
  .pipe(lambda x: x.style.format(dict(id=embed_spotify)))
)